# Listen to Kafka topic and reproduce processed video

### Using kafka-python and OpenCV
![kafka](http://www.godatafy.com/wp-content/uploads/2015/07/tech-apachekafka-logo.jpg) ![opencv](http://www.decom.ufop.br/imobilis/wp-content/uploads/2015/03/OpenCV_Logo.png) ![python](https://jsantanna.files.wordpress.com/2016/07/python-logo.png)

In [8]:
# Import video, kafka and other libs
from kafka import KafkaConsumer
import numpy as np
import cv2
import pickle, base64, json
from datetime import datetime, date, time

In [9]:
# Setup kafka broker and topic
kafkabrokers='wn0-kafka2.e4ix3a2tssaedkygl5ugloqjrb.cx.internal.cloudapp.net:9092'
topic='camera-analytics-stream'
cameraId=0

# Subscribe to topic
consumer = KafkaConsumer(bootstrap_servers=kafkabrokers, api_version=(0,10),
                        value_deserializer=lambda v: json.loads(v.decode('utf-8')))
consumer.subscribe([topic])

In [10]:
# Recreate video stream from camera with analytics on it
for kafka_msg in consumer:
    # get only the cameradId we want to watch
    if(kafka_msg.value['cameraId'] == cameraId):
        # Log delay from video frame taken and frame processed arrived
        print('\rframe delay =', datetime.utcnow()-datetime.strptime(str(kafka_msg.value['frameTS']), "%Y-%m-%d %H:%M:%S.%f"), end='')
        print(' / face count =', kafka_msg.value['facesCount'], end='')

        # Extract frame back from kafka message
        frame = pickle.loads(base64.standard_b64decode(kafka_msg.value['frameB64'].encode('utf-8')))
        # Show frames as they arrive
        cv2.imshow('faces', frame)
    # look for a signal to stop capturing video
    if cv2.waitKey(5) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

frame delay = 0:00:01.910615 / face count = 1